<a href="https://colab.research.google.com/github/MO230101/The-codes-for-hydrogel-study-/blob/main/Mahalanobis_distance_for_paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import mahalanobis
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl

# CSVファイルを読み込む
data = pd.read_csv('file name.csv', index_col=0)

# データの表示（最初の数行）
print("読み込んだデータ:")
print(data.head())

# データ前処理（数値データのみに絞る）
numeric_data = data.select_dtypes(include=[np.number])

# 最初の列を基準として設定
base_column = numeric_data.iloc[:, 0]

# 最初の列と他の各列ごとに処理を繰り返す
for col_index in range(1, numeric_data.shape[1]):
    target_column = numeric_data.iloc[:, col_index]

    # 基準列とターゲット列のデータフレームを作成
    subset = pd.DataFrame({
        numeric_data.columns[0]: base_column,
        numeric_data.columns[col_index]: target_column
    }, index=data.index)  # インデックスを保持

    # マハラノビス距離を計算
    cov_matrix = np.cov(subset.T)  # 共分散行列を計算
    inv_cov_matrix = np.linalg.inv(cov_matrix)  # 逆共分散行列を計算
    mean_vector = subset.mean(axis=0).values  # 平均を計算

    # 各データポイントのマハラノビス距離を計算
    distances = subset.apply(
        lambda row: mahalanobis(row, mean_vector, inv_cov_matrix), axis=1
    )

    # 結果をデータフレームに追加
    subset['Mahalanobis_Distance'] = distances

    # 散布図を作成
    plt.figure(figsize=(8, 6))
    scatter = sns.scatterplot(
        data=subset,
        x=subset.columns[0],  # x軸（基準列）
        y=subset.columns[1],  # y軸（ターゲット列）
        hue='Mahalanobis_Distance',  # 色付け
        palette='viridis',
        size='Mahalanobis_Distance',  # 点のサイズ
        sizes=(20, 200)
    )

    # マハラノビス距離が4以上のデータポイントに名前を付ける
    for i in range(len(subset)):
        if subset.iloc[i]['Mahalanobis_Distance'] >= 2:  # 閾値を4に設定
            plt.text(
                x=subset.iloc[i, 0],  # x座標（基準列の値）
                y=subset.iloc[i, 1],  # y座標（ターゲット列の値）
                s=subset.index[i],   # インデックスを表示
                fontsize=9, color='red', weight='bold'
            )

    # 相関係数を計算
    correlation = subset.iloc[:, 0].corr(subset.iloc[:, 1])

    # 相関係数をプロットに追加（凡例と重ならないように配置）
    plt.text(
        0.85, 0.95, f'Correlation: {correlation:.2f}',
        ha='right', va='top', transform=plt.gca().transAxes,
        fontsize=12, bbox=dict(facecolor='white', edgecolor='None', alpha=0.7)
    )

    # カラーバーを追加
    norm = mpl.colors.Normalize(vmin=subset['Mahalanobis_Distance'].min(), vmax=subset['Mahalanobis_Distance'].max())
    sm = plt.cm.ScalarMappable(cmap="viridis", norm=norm)
    sm.set_array([])
    plt.colorbar(sm, ax=plt.gca(), label="Mahalanobis Distance")

    # プロット設定
    plt.title(f"Scatter Plot of {numeric_data.columns[0]} vs {numeric_data.columns[col_index]} with Mahalanobis Distance")
    plt.xlabel(numeric_data.columns[0])  # x軸ラベル（基準列名）
    plt.ylabel(numeric_data.columns[col_index])  # y軸ラベル（ターゲット列名）
    plt.tight_layout()
    plt.show()

    # マハラノビス距離の結果を表示
    print(f"\nMahalanobis距離({numeric_data.columns[0]} vs {numeric_data.columns[col_index]}）:")
    print(subset[['Mahalanobis_Distance']])
